# One class SVM model for intruder detection
This notebook contains code to run experiments with a 1-class SVM model for user authentiation. 

The experiments run multithreaded and output the result to a specified directory. You can change the output directory by changing the variable `OUTPUT_DIR` variable, and set the number of threads using the `N_THREADS` variable. Using more threads will increase memory demands, so limit to 1 thread/GB of avilable memory.

In [1]:
import multiprocessing as mp
import os
import pickle
import time

import numpy as np
import sklearn.metrics
from sklearn.model_selection import ParameterGrid, TimeSeriesSplit
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM
from tqdm import tqdm

from config import *

%matplotlib inline
%load_ext autoreload
%autoreload 2


## Parameters for the experiments
Set the parameters with which the experiment will be run. Below is a description for each of them.
* `DATASET`: Specify which dataset to be used. Choose between BRAINRUN and TOUCHALYTICS. The paths for the dataset are assumed to be at `./datasets`, and can be changed in the `config.py` file.
* `N_THREADS`: Number of threads to use for the experiment.
* `OUTPUT_DIR`: Directory to which the results will be written.
* `USE_PRESSURE_AND_AREA_FEATURES`: Whether to use pressure and area features - only for the *Touchalytics* dataset, is ignored when using the *BrainRun* dataset.

Here we also set the parameters for data cleanup and parsing, although these are best left constant.
* `MIN_SESSION_GESTURES`: Minimum number of gestures in a session to be considered. Left at 140 as it has shown to produce good results, and include a large number of users
* `SCREENS`: The screens to use from the BrainRun dataset when performing the experiment (ignored for the Touchalytics dataset). The experiments were originally performed using either one or both of the screens *MathisisGame* or *FocusGame*, as they contain predominantly swipe data.
* `WINDOW_SIZE`: Left at 1 constant throught these experiments, as majority voting over sliding windows is employed later to verify the results. 


In [2]:
DATASET = TOUCHALYTICS # Choose between BRAINRUN and TOUCHALYTICS
USE_PRESSURE_AND_AREA_FEATURES = True
N_THREADS = 32
OUTPUT_DIR = 'test_results/'

MIN_SESSION_GESTURES = 140
SCREENS = ['MathisisGame', 'FocusGame']
WINDOW_SIZE = 1

## Utility methods for extracting features and splitting the data

In [3]:
def extract_features(data, delta_time):
    '''
    Extracts manually  engineered features from a gesture (swipe). 
    
    Returns an array of length 23 (25 if area and pressure are used) with
    the extracted features
    '''
    result = []
    points = [(data[0]['x0'], data[0]['y0'])] + [(x['moveX'], x['moveY']) for x in data]
    points = np.array(points)

    # End to end distance - from paper (Touchalytics) and https://par.nsf.gov/servlets/purl/10167262
    result.append(np.linalg.norm(points[-1] - points[0]))
    # Path length - from Touchalytics adn https://par.nsf.gov/servlets/purl/10167262
    result.append(sum(np.linalg.norm(points[i] - points[i - 1]) for i in range(1, len(points))))
    # Path length / delta time - from https://par.nsf.gov/servlets/purl/10167262
    result.append(result[-1] / delta_time)
    # Average Acceleration
    # result.append(result[-1] / delta_time)

    # 20th percentile velocity - Touchalytics
    result.append(
        np.percentile(
            [np.linalg.norm(np.array([current_point['vx'], current_point['vy']])) for current_point in data],
            20
        )
    )

    # 80th percentile velocity - Touchalytics
    result.append(
        np.percentile(
            [np.linalg.norm(np.array([current_point['vx'], current_point['vy']])) for current_point in data],
            80
        )
    )

    # Angle (direction of end-to-end line) - Touchalytics
    result.append(np.angle(
        complex(points[-1][0] - points[0][0], points[-1][1] - points[0][1])
    ))

    # Largest distance from end-to-end line (signed) - Touchalytics
    result.append(
        np.max(
            [np.cross(points[-1] - points[0], points[0] - p) / (np.linalg.norm(points[-1] - points[0]) or 1) for p in points]
        ))

    # Largest deviation from end to end line
    result.append(max(abs(np.arctan2(*(points[i] - points[i - 1]))) for i in range(1, len(points))))

    # First 3 point acceleration
    result.append(np.linalg.norm(2 * points[1] - points[0] - points[2]) / 2 if len(points) > 3 else 0)
    # Last 3 point acceleration
    result.append(np.linalg.norm(2 * points[-2] - points[-3] - points[-1]) / 2 if len(points) > 3 else 0)

    # Median velocity at last three points - Touchalytics (11 features + duration = 12 - without the first and last 3 points)
    result.append(
        np.median(
            np.array(
                [np.linalg.norm(np.array([data[i]['vx'], data[i]['vy']])
                ) for i in range(len(data) - 1, max(-1, len(data) - 4), -1)])))

    # Only for touchalytics
    if USE_PRESSURE_AND_AREA_FEATURES and DATASET == TOUCHALYTICS:
        # Midstroke area covered
        result.append(data[int(len(data) // 2)]['area'])
        # Midstroke pressure
        result.append(data[int(len(data) // 2)]['pressure'])

    res = np.nan_to_num(result)

    first_three_points = points[:3].flatten() 
    last_three_points = points[-3:].flatten() 

    first_three_points.resize((6,))
    last_three_points.resize((6,))

    return np.concatenate([res, first_three_points, last_three_points])
    
def gesture_to_data(c):
    # Stroke duration
    delta_time = (c['t_stop'] - c['t_start']) / 1000
    extra_features = extract_features(c['data'], delta_time)

    # Start and stop time are removed when sliding windows is called, and transformed in relative time
    # between the gestures in a window
    return np.concatenate([[c['t_start'], c['t_stop'], delta_time], extra_features])

def window_to_datapoint(window):
    return np.concatenate([
        window[:, 2:].flatten(), # Exclude start and stop time
        (window[1:, 0] - window[0, 1]).flatten() / 1000, # Window start - initial point stop (time from window start for each gesture)
        (window[1:, 0] - window[:-1, 1]).flatten() / 1000]) # Window start - previous window stop (pairwise time between gestures)

def session_to_datapoints(s):
    '''
    Converts a session to a series of datapoints. Also slides windows of length WINDOW_SIZE over the gestures
    and considers each window as a datapoint. 
    '''
    featurized_session = np.array([gesture_to_data(x) for x in s['gestures']])
    sliding_windows = (
        np.expand_dims(np.arange(WINDOW_SIZE), 0) +
        np.expand_dims(np.arange(len(featurized_session) - WINDOW_SIZE), 0).T
    )

    return np.array([window_to_datapoint(window) for window in featurized_session[sliding_windows]])

def get_train_indices(size, test_size = 0.2, gap = WINDOW_SIZE, max_size = np.inf):
    '''
    Returns the train indices for a given session.
    Leaves a space of `gap` between the train and test indices
    '''
    size = min(size, max_size)
    middle = int(size * (1 - test_size))
    middle = min(middle, max_size)
    return np.arange(middle - gap)

def get_test_indices(size, test_size = 0.2, gap = WINDOW_SIZE, max_size = np.inf):
    '''
    Returns the test indices for a given session.
    '''
    size = min(size, max_size)
    middle = int(size * (1 - test_size))
    return np.arange(middle, size)

def get_intruder_size(size, test_size = 0.2, gap = WINDOW_SIZE, max_size = np.inf):
    '''
    Returns validation and test indices for intruders. Also leaves a space of `gap` 
    between the validation and test indices.
    '''
    size = min(size, max_size)
    middle = int(size * (1 - test_size))
    validation_middle = int(size * (1 - test_size / 2))
    return np.arange(middle, validation_middle - gap), np.arange(validation_middle, size) 

## Methods for filtering and parsing the data

In [4]:
def prefilter_session(s):
    '''
    Filters the session, orders gestures chronologically and removes gestures that are outliers or from different screens
    '''
    s['gestures'].sort(key = lambda x: x['t_start'])
    s['gestures'] = [x for x in s['gestures'] 
        if x['t_stop'] - x['t_start'] > 70 and x['t_stop'] - x['t_start'] < (1000 if DATASET == BRAINRUN else 2000) and 
        ((x['screen'].split(' ')[0] in SCREENS and x['type'] == 'swipe') if DATASET == BRAINRUN else True)]

def parse_user(user_id):
    '''
    Parses all the sessions for a user with the given id. Deletes sessions that are too short after filtering them.
    '''
    i = 0
    should_delete = False
    while i < len(users[user_id]['devices'][0]['sessions']):
        prefilter_session(users[user_id]['devices'][0]['sessions'][i])

        if len(users[user_id]['devices'][0]['sessions'][i]['gestures']) < MIN_SESSION_GESTURES or should_delete:
            del users[user_id]['devices'][0]['sessions'][i]
        else:
            users[user_id]['devices'][0]['sessions'][i] = session_to_datapoints(users[user_id]['devices'][0]['sessions'][i])
            # Outlier detection
            clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)
            users[user_id]['devices'][0]['sessions'][i] = \
                users[user_id]['devices'][0]['sessions'][i][np.where(clf.fit_predict(users[user_id]['devices'][0]['sessions'][i]) == 1)]
            i += 1

def get_users_over_gestures(number_of_gestures = 140):
    '''
    Returns an array with the indices of all users with more than number_of_gestures gestures.
    '''
    uc = np.zeros((len(users), ))
    for i in range(len(users)):
        uc[i] = 0
        for session in users[i]['devices'][0]['sessions']:
            uc[i] += session.shape[0]

    return np.where(uc > number_of_gestures)[0]

def compute_eer(label, pred):
    """
    Computes EER given a list of labels and predictions.

    Code inspired by https://github.com/YuanGongND/python-compute-eer
    """
    # all fpr, tpr, fnr, fnr, threshold are lists (in the format of np.array)
    fpr, tpr, threshold = sklearn.metrics.roc_curve(label, pred)
    fnr = 1 - tpr

    # theoretically eer from fpr and eer from fnr should be identical but they can be slightly differ in reality
    eer_1 = fpr[np.nanargmin(np.absolute((fnr - fpr)))]
    eer_2 = fnr[np.nanargmin(np.absolute((fnr - fpr)))]

    # return the mean of eer from fpr and from fnr
    eer = (eer_1 + eer_2) / 2
    return eer


# Code for multithreaded experiments

Since the datasets are large and a iteration is done for each users, the experiments take a significant amount of time to run on a normal machine. Multiprocessing was used to run the experiments in parallel on a powerful machine, using 32 cores (this reduces the time to about an hour). Below is the code used to run the experiments using different processes. 

In [5]:
# Parameter space (for hyperparameter tuning)
parameters = [{
    'kernel': ['rbf'], 'gamma': [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001], 'nu': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],},
    ]

def run_experiment_multithreaded(output_path, dataset, valid_users):
    FILE_PATH = f'{output_path}/{dataset}'

    def run_experiment_for_user(user_id, users, valid_users, parameters):
        max_train_per_session = np.inf

        # Uncomment this line to test the effect of different number of gestures
        # max_train_per_session = int(140 / len(users[int(user_id)]['devices'][0]['sessions']))
        
        X_train = np.concatenate([session[get_train_indices(session.shape[0], max_size=max_train_per_session)] for session in users[int(user_id)]['devices'][0]['sessions']])
        X_test = np.concatenate([session[get_test_indices(session.shape[0]),] for session in users[int(user_id)]['devices'][0]['sessions']])
        
        # Cross validation - choose best hyperparameters for each user
        # Default hyperparameters (will be replaced by the best hyperparameters)
        hyperparameters = [(100, {'kernel':'rbf', 'gamma':0.8, 'nu': 0.3})]
        for hyper_pair in ParameterGrid(parameters):
            # Use time series split cross-validation
            cv = TimeSeriesSplit(n_splits=4, gap = WINDOW_SIZE)
            avg_eer = []
            for train, test in cv.split(X_train):
                clf = OneClassSVM(
                    kernel = hyper_pair['kernel'], 
                    nu=hyper_pair['nu'], 
                    degree = hyper_pair['degree'] if 'degree' in hyper_pair else 0, 
                    gamma = hyper_pair['gamma'] if 'gamma' in hyper_pair else 0,)

                scaler = StandardScaler()
                clf.fit(scaler.fit_transform(X_train[train]))

                # Predict
                res = np.concatenate([
                    clf.decision_function(scaler.transform(X_train[test])), *[
                        clf.decision_function(scaler.transform(session[get_intruder_size(session.shape[0])[0]])) for other_uid in valid_users[valid_users != user_id] for session in users[other_uid]['devices'][0]['sessions']]
                ])
                # Build labels based on test data and result
                y_test = np.concatenate([np.zeros((test.shape[0],)) + 1, np.zeros((res.shape[0] - test.shape[0],)) - 1])
                
                # Set inf and -inf predictions to a reasonable number
                res[np.isneginf(res)] = -100000
                res[np.isposinf(res)] = 100000
                avg_eer.append(compute_eer(y_test, res))
            hyperparameters.append((np.mean(avg_eer), hyper_pair))
        
        # Get the best hyperparams
        best_hyperparameters = sorted(hyperparameters, key=lambda x: x[0])[0][1]
        svm = OneClassSVM(
                    kernel = best_hyperparameters['kernel'], 
                    nu=best_hyperparameters['nu'], 
                    degree = best_hyperparameters['degree'] if 'degree' in best_hyperparameters else 0, 
                    gamma = best_hyperparameters['gamma'] if 'gamma' in best_hyperparameters else 0,)

        # Normalize the data using StandardScaler - fit only on train data and use the same scaler for both train and test
        scaler = StandardScaler()
        svm.fit(scaler.fit_transform(X_train))

        # Predict
        res = np.concatenate([
            svm.decision_function(scaler.transform(X_test)), *[
                svm.decision_function(scaler.transform(session[get_intruder_size(session.shape[0])[1]])) for other_uid in valid_users[valid_users != user_id] for session in users[other_uid]['devices'][0]['sessions']]
        ])

        # Build labels based on the test data and result - user labels are set as 1, intruder labels are set as -1
        y_test = np.concatenate([np.zeros((X_test.shape[0],)) + 1, np.zeros((res.shape[0] - X_test.shape[0],)) - 1])

        # Save results with pickle to a file
        with open(f'{FILE_PATH}/user_{user_id}.pkl', 'wb') as f:
            pickle.dump((y_test, res, hyperparameters), f)

        # print(compute_eer(y_test, res))


        # results[user_id]['eer'] = compute_eer(y_test, res)

    def find_hyper(user_id, hyper_pair, users, valid_users, X_train, hyperparameters):
        cv = TimeSeriesSplit(n_splits=4, gap = WINDOW_SIZE)
        avg_eer = []
        for train, test in cv.split(X_train):
            clf = OneClassSVM(
                kernel = hyper_pair['kernel'], 
                nu=hyper_pair['nu'], 
                degree = hyper_pair['degree'] if 'degree' in hyper_pair else 0, 
                gamma = hyper_pair['gamma'] if 'gamma' in hyper_pair else 0,)
            scaler = StandardScaler()
            clf.fit(scaler.fit_transform(X_train[train]))
            res = np.concatenate([
                clf.decision_function(scaler.transform(X_train[test])), *[
                    clf.decision_function(scaler.transform(session[get_intruder_size(session.shape[0])[0]])) for other_uid in valid_users[valid_users != user_id] for session in users[other_uid]['devices'][0]['sessions']]
            ])
            y_test = np.concatenate([np.zeros((test.shape[0],)) + 1, np.zeros((res.shape[0] - test.shape[0],)) - 1])
            
            res[np.isneginf(res)] = -1000
            res[np.isposinf(res)] = 1000
            avg_eer.append(compute_eer(y_test, res))
        hyperparameters.append((np.mean(avg_eer), hyper_pair))

    # results = [{i: {}} for i in range(300)]

    if not os.path.exists(FILE_PATH):
        os.makedirs(FILE_PATH)

    # ERROR HERE
    for user_id in list(set(valid_users).intersection([0,1] if DATASET == BRAINRUN else [])):
        max_train_per_session = np.inf

        # Uncomment this line to test the effect of different number of gestures
        # max_train_per_session = int(140 / len(users[int(user_id)]['devices'][0]['sessions']))
        
        X_train = np.concatenate([session[get_train_indices(session.shape[0], max_size=max_train_per_session)] for session in users[int(user_id)]['devices'][0]['sessions']])
        X_test = np.concatenate([session[get_test_indices(session.shape[0]),] for session in users[int(user_id)]['devices'][0]['sessions']])

        # Cross validation
        # Users 0 and 1 of the BrainRun dataset contain significantly more data, and cross-validation is done multi-threaded for efficiency.
        # For all other users cross-validation is done in a single thread, as the overhead is not justified.
        threads = []
        cid = 0
        can_exit = False

        hyper_grid = list(ParameterGrid(parameters))
        manager = mp.Manager()
        hyperparameters = manager.list()

        pbar = tqdm(total=len(hyper_grid))
        while not can_exit:
            while len(threads) < N_THREADS and cid < len(hyper_grid):
                thread = mp.Process(target=find_hyper, args=(user_id, hyper_grid[cid], users, valid_users, X_train, hyperparameters))
                thread.start()
                threads.append(thread)
                pbar.update(1)
                cid += 1

            for thread in threads:
                if not thread.is_alive():
                    thread.join()
                    threads.remove(thread)

            if(len(threads) == 0):
                can_exit = True
            time.sleep(1)
        pbar.close()

        # End hyperparameter search

        # Fit and test the model (same process as before)
        hyperparameters = list(hyperparameters)
        best_hyperparameters = sorted(hyperparameters, key=lambda x: x[0])[0][1]
        svm = OneClassSVM(
                    kernel = best_hyperparameters['kernel'], 
                    nu=best_hyperparameters['nu'], 
                    degree = best_hyperparameters['degree'] if 'degree' in best_hyperparameters else 0, 
                    gamma = best_hyperparameters['gamma'] if 'gamma' in best_hyperparameters else 0,)
        scaler = StandardScaler()

        svm.fit(scaler.fit_transform(X_train))

        res = np.concatenate([
            svm.decision_function(scaler.transform(X_test)), *[
                svm.decision_function(scaler.transform(session[get_intruder_size(session.shape[0])[1]])) for other_uid in valid_users[valid_users != user_id] for session in users[other_uid]['devices'][0]['sessions']]
        ])

        y_test = np.concatenate([np.zeros((X_test.shape[0],)) + 1, np.zeros((res.shape[0] - X_test.shape[0],)) - 1])

        # Save results with pickle to a file
        with open(f'{FILE_PATH}/user_{user_id}.pkl', 'wb') as f:
            pickle.dump((y_test, res, hyperparameters), f)

    threads = []
    cid = 0
    can_exit = False

    if not os.path.exists(FILE_PATH):
        os.makedirs(FILE_PATH)

    # Same process of excluding the first two users for the BrainRun dataset
    vu = list(set(valid_users).difference([0,1] if DATASET == BRAINRUN else []))
    pbar = tqdm(total=len(vu))
    while not can_exit:
        while len(threads) < N_THREADS and cid < len(vu):
            user_id = vu[cid]
            thread = mp.Process(target=run_experiment_for_user, args=(vu[cid], users, valid_users, parameters))
            thread.start()
            threads.append(thread)
            cid += 1
            pbar.update(1)

        for thread in threads:
            if not thread.is_alive():
                thread.join()
                threads.remove(thread)

        if(len(threads) == 0):
            can_exit = True

        time.sleep(1)
    

## Run experiments with the set parameters|

In [6]:
if DATASET == BRAINRUN:
    with open(f'{DATA_PATH}/brainrun_full_not_parsed.pkl', 'rb') as f:
        users = pickle.load(f)

if DATASET == TOUCHALYTICS:
    with open(f'{DATA_PATH}/touchalytics_full_not_parsed.pkl', 'rb') as f:
        users = pickle.load(f)

for user in tqdm(range(len(users))):
    parse_user(user)

valid_users = get_users_over_gestures(140)

# Uncomment the following lines to see the effect of using less users
# These experiments were performed multiple times, and the results were averaged
# np.random.shuffle(valid_users)
# valid_users = valid_users[:<MAX_USER_SIZE>]

run_experiment_multithreaded(OUTPUT_DIR, DATASET, valid_users)


 41%|████▏     | 17/41 [00:20<00:28,  1.18s/it]


KeyboardInterrupt: 